<a href="https://colab.research.google.com/github/zubaerimran/219_projs/blob/master/code/gan/gan_svhn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Semi-Supervised Learning GAN - MNIST dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import time
import os
import pdb

import tensorflow as tf
from tensorflow.contrib.layers.python.layers import batch_norm

from sklearn.metrics import roc_auc_score

from matplotlib import pyplot as plt
#%matplotlib inline

Initialization

In [0]:
x_height, x_width = [32, 32]
num_channels = 3
num_classes = 10
latent_size = 100
labeled_rate = 0.1
g_scale_factor = 1 - 0.75/2
d_scale_factor = 0.25
eps_min = 1e-8 # used to avoid NAN loss
eps_max = 1.0
alpha = 0.2

save_to = 'gdrive/My Drive/gan/svhn/model_Jan16/out/'

if not os.path.exists(save_to):
    os.makedirs(save_to)

log_path = 'gdrive/My Drive/gan/svhn/model_Jan16/gan_svhn_log.csv'
model_path ='gdrive/My Drive/gan/svhn/model_Jan16/gan_svhn_model.ckpt'

In [0]:
# norm_tr1 = (norm_tr + 1.0)/2.0
# np.max(norm_tr), np.max(norm_tr1)

In [0]:
# #checking training data
# tr = dataset['train_images']


# norm_tr = normalize(tr[:256])
# plot(norm_tr1[:16])

In [0]:
def normalize(x):
    # normalize data
    x = (x - 127.5) / 127.5
    return x.reshape((-1, x_height, x_width, num_channels))

In [0]:
import scipy.io as sio
dataset = sio.loadmat('gdrive/My Drive/aelan_datasetes/svhn.mat')

In [0]:
# plot_fake_data(norm_tr1)

In [0]:
# #checking training data
# tr = dataset['train_images']
# plot(tr[:64])

# norm_tr = normalize(tr)
# norm_tr1 = (norm_tr +1)/2
# plot(norm_tr1[:64])

In [0]:
# def G(inputs, is_training, reuse = False, print_summary = False):
#     with tf.variable_scope('Generator', reuse = reuse) as scope:

#         normal_initializer = tf.random_normal_initializer(mean=0.0, stddev=0.02)
        
#         # Expand input to [batch, 1, 1, channels]
#         inputs = tf.expand_dims(tf.expand_dims(inputs, 1), 1)
        
#         # Transposed convolution outputs [batch, 4, 4, 1024]
#         net = tf.layers.conv2d_transpose(inputs, 
#                                          filters=1024, 
#                                          kernel_size=4, 
#                                          padding='valid',
#                                          kernel_initializer=normal_initializer,
#                                          trainable=is_training,
#                                          name='tconv1')

#         net = tf.layers.batch_normalization(net, 
#                                             training=is_training,
#                                             name='tconv1/batch_normalization')

#         net = tf.nn.relu(net, name='tconv1/relu')
        
#         # Transposed convolution outputs [batch, 8, 8, 256]
#         net = tf.layers.conv2d_transpose(net, 
#                                          filters=256, 
#                                          kernel_size=4, 
#                                          strides=2, 
#                                          padding='same',
#                                          kernel_initializer=normal_initializer,
#                                          trainable=is_training,
#                                          name='tconv2')

#         net = tf.layers.batch_normalization(net, 
#                                             training=is_training,
#                                             name='tconv2/batch_normalization')
#         net = tf.nn.relu(net, name='tconv2/relu')
        
#         # Transposed convolution outputs [batch, 16, 16, 64]
#         net = tf.layers.conv2d_transpose(net, 
#                                          filters=64, 
#                                          kernel_size=4, 
#                                          strides=2, 
#                                          padding='same',
#                                          kernel_initializer=normal_initializer,
#                                          trainable=is_training,
#                                          name='tconv3')

#         net = tf.layers.batch_normalization(net, 
#                                             training=is_training,
#                                             name='tconv3/batch_normalization')
#         net = tf.nn.relu(net, name='tconv3/relu')
        
#         # Transposed convolution outputs [batch, 32, 32, 3]
#         net = tf.layers.conv2d_transpose(net, 
#                                          filters=3, 
#                                          kernel_size=4, 
#                                          strides=2, 
#                                          padding='same',
#                                          kernel_initializer=normal_initializer,
#                                          trainable=is_training,
#                                          name='tconv4')

#         net = tf.tanh(net, name='tconv4/tanh')
        
#         return net

# def D(inputs, dropout_rate, is_training, reuse = True, print_summary = True):

#   with tf.variable_scope('Discriminator', reuse = reuse) as scope:      

#         normal_initializer = tf.random_normal_initializer(mean=0.0, stddev=0.02)
        
#         # Convolution outputs [batch, 16, 16, 64]
#         net = tf.layers.conv2d(inputs, 
#                                filters=64, 
#                                kernel_size=4, 
#                                strides=2, 
#                                padding='same',
#                                kernel_initializer=normal_initializer,
#                                trainable=is_training,
#                                name='conv1')

#         net = leaky_relu(net, 0.2, name='conv1/leaky_relu')
        
#         # Convolution outputs [batch, 8, 8, 256]
#         net = tf.layers.conv2d(net, 
#                                filters=256, 
#                                kernel_size=4, 
#                                strides=2, 
#                                padding='same',
#                                kernel_initializer=normal_initializer,
#                                trainable=is_training,
#                                name='conv2')

#         net = tf.layers.batch_normalization(net, 
#                                             training=is_training,
#                                             name='conv2/batch_normalization')

#         net = leaky_relu(net, 0.2, name='conv2/leaky_relu')
        
#         # Convolution outputs [batch, 4, 4, 1024]
#         net = tf.layers.conv2d(net, 
#                                filters=1024, 
#                                kernel_size=4, 
#                                strides=2, 
#                                padding='same',
#                                kernel_initializer=normal_initializer,
#                                trainable=is_training,
#                                name='conv3')

#         net = tf.layers.batch_normalization(net, 
#                                             training=is_training,
#                                             name='conv3/batch_normalization')

#         net = leaky_relu(net, 0.2, name='conv3/leaky_relu')
        
#         # Convolution outputs [batch, 1, 1, 1]
# #         net = tf.layers.conv2d(net, 
# #                                filters=1, 
# #                                kernel_size=4, 
# #                                padding='valid',
# #                                kernel_initializer=normal_initializer,
# #                                trainable=is_training,
# #                                name='conv4')
        
#         # Squeeze height and width dimensions
#         #net = tf.squeeze(net, [1, 2, 3])
        
#         flatten_length = lrelu4.get_shape().as_list()[1] * \
# #                          lrelu4.get_shape().as_list()[2] * lrelu4.get_shape().as_list()[3]
# #         flatten5 = tf.reshape(lrelu4, (-1, flatten_length)) # used for "Feature Matching" 
# #         fc5 = tf.layers.dense(flatten5, (num_classes + 1))
# #         output = tf.nn.softmax(fc5)

# #         assert output.get_shape()[1:] == [num_classes + 1]

# #         if print_summary:
# #             print('Discriminator summary:\n x: %s\n' \
# #                   ' D1: %s\n D2: %s\n D3: %s\n D4: %s\n' %(x.get_shape(), 
# #                                                            dropout1.get_shape(),
# #                                                            lrelu2.get_shape(), 
# #                                                            dropout3.get_shape(),
# #                                                            lrelu4.get_shape()))
# #         return flatten5, fc5, output

In [0]:
def D(x, dropout_rate, is_training, reuse = True, print_summary = True):
    # discriminator (x -> n + 1 class)

    with tf.variable_scope('Discriminator', reuse = reuse) as scope:
        normal_initializer = tf.random_normal_initializer(mean=0.0, stddev=0.02)
        # layer1 - do not use Batch Normalization on the first layer of Discriminator
        conv1 = tf.layers.conv2d(x, 64, [5, 5],
                                 strides = [2, 2],
                                 padding = 'same',
                                 kernel_initializer=normal_initializer,)
        lrelu1 = tf.maximum(0.2 * conv1, conv1) #leaky relu
        dropout1 = tf.layers.dropout(lrelu1, dropout_rate)

        # layer2
        conv2 = tf.layers.conv2d(dropout1, 128, [3, 3],
                                 strides = [2, 2],
                                 padding = 'same',
                                 kernel_initializer=normal_initializer)
        batch_norm2 = tf.layers.batch_normalization(conv2, training = is_training)
        lrelu2 = tf.maximum(0.2 * batch_norm2, batch_norm2)

        # layer3
        conv3 = tf.layers.conv2d(lrelu2, 256, [2, 2],
                                 strides = [2, 2],
                                 padding = 'same',
                                 kernel_initializer=normal_initializer)
        batch_norm3 = tf.layers.batch_normalization(conv3, training = is_training)
        lrelu3 = tf.maximum(0.2 * batch_norm3, batch_norm3)
        dropout3 = tf.layers.dropout(lrelu3, dropout_rate)

        # layer 4
        conv4 = tf.layers.conv2d(dropout3, 512, [2, 2],
                                 strides = [2, 2],
                                 padding = 'same',
                                 kernel_initializer=normal_initializer)
        # do not use batch_normalization on this layer - next layer, "flatten5",
        # will be used for "Feature Matching"
        lrelu4 = tf.maximum(0.2 * conv4, conv4)

        # layer 5
        flatten_length = lrelu4.get_shape().as_list()[1] * \
                         lrelu4.get_shape().as_list()[2] * lrelu4.get_shape().as_list()[3]
        flatten5 = tf.reshape(lrelu4, (-1, flatten_length)) # used for "Feature Matching" 
        fc5 = tf.layers.dense(flatten5, (num_classes + 1))
        output = tf.nn.softmax(fc5)

        assert output.get_shape()[1:] == [num_classes + 1]

        if print_summary:
            print('Discriminator summary:\n x: %s\n' \
                  ' D1: %s\n D2: %s\n D3: %s\n D4: %s\n' %(x.get_shape(), 
                                                           dropout1.get_shape(),
                                                           lrelu2.get_shape(), 
                                                           dropout3.get_shape(),
                                                           lrelu4.get_shape()))
        return flatten5, fc5, output

In [0]:
alpha = 0.2
def G(z, is_training, reuse = False, print_summary = False):
    # generator (z -> x)

    with tf.variable_scope('Generator', reuse = reuse) as scope:
        normal_initializer = tf.random_normal_initializer(mean=0.0, stddev=0.02)
        # layer 0
        z_ = tf.layers.dense(z, 512*2*2)
        z_reshaped = tf.reshape(z_, (-1, 2, 2, 512))
        z_dense = tf.nn.relu(z_reshaped)

        # layer 1 out: 8x8
        deconv1 = tf.layers.conv2d_transpose(z_dense,
                                             filters = 256,
                                             kernel_size = [5, 5],
                                             strides = [2, 2],
                                             padding = 'same',
                                             kernel_initializer=normal_initializer)
        batch_norm1 = tf.layers.batch_normalization(deconv1, training = is_training)
        relu1 = tf.nn.relu(batch_norm1)
        
        
        
        # layer 2 out: 16x16
        deconv2 = tf.layers.conv2d_transpose(relu1,
                                             filters = 128,
                                             kernel_size = [5, 5],
                                             strides = [2, 2],
                                             padding = 'same',
                                             kernel_initializer=normal_initializer)
        batch_norm2 = tf.layers.batch_normalization(deconv2, training = is_training)
        relu2 = tf.nn.relu(batch_norm2)
        
        
        # layer 3 out:32x32
        deconv3 = tf.layers.conv2d_transpose(relu2,
                                             filters = 64,
                                             kernel_size = [5, 5],
                                             strides = [2, 2],
                                             padding = 'same',
                                             kernel_initializer=normal_initializer)
        batch_norm3 = tf.layers.batch_normalization(deconv3, training = is_training)
        relu3 = tf.nn.relu(batch_norm3)

        
#         # layer 4 out: 64x64
#         deconv4 = tf.layers.conv2d_transpose(lrelu3,
#                                              filters = 64,
#                                              kernel_size = [3, 3],
#                                              strides = [2, 2],
#                                              padding = 'same')
#         batch_norm4 = tf.layers.batch_normalization(deconv4, training = is_training)
#         lrelu4 = tf.nn.leaky_relu(batch_norm4, alpha)
        
        
#         # layer 5 out: 128x128
#         deconv5 = tf.layers.conv2d_transpose(lrelu4,
#                                              filters = 32,
#                                              kernel_size = [3, 3],
#                                              strides = [2, 2],
#                                              padding = 'same')
#         batch_norm5 = tf.layers.batch_normalization(deconv4, training = is_training)
#         lrelu5 = tf.nn.leaky_relu(batch_norm5, alpha)
        
        
        # layer 5 - do not use Batch Normalization on the last layer of Generator
        deconv4 = tf.layers.conv2d_transpose(relu3,
                                             filters = num_channels,
                                             kernel_size = [3, 3],
                                             strides = [2, 2],
                                             padding = 'same',
                                             kernel_initializer=normal_initializer)
        tanh4 = tf.tanh(deconv4)
        
        print(tanh4.shape)
                                             
                                             
        assert tanh4.get_shape()[1:] == [x_height, x_width, num_channels]
        
        if print_summary:
            print('Generator summary:\n z: %s\n' \
                  ' G0: %s\n G1: %s\n G2: %s\n G3: %s\n G4: %s\n' %(z_.get_shape(),
                                                                    z_reshaped.get_shape(),
                                                                    relu1.get_shape(),
                                                                    relu2.get_shape(),
                                                                    relu3.get_shape(),
                                                                    #lrelu4.get_shape(),
                                                                    tanh4.get_shape()))
        return tanh4

In [0]:
#For single D
def build_model(x_real, z, label, dropout_rate, is_training, print_summary = False):
    # build model
    D_real_features, D_real_logit, D_real_prob = D(x_real, dropout_rate, is_training,
                                                   reuse = False, print_summary = print_summary)
    x_fake = G(z, is_training, reuse = False, print_summary = print_summary)
    D_fake_features, D_fake_logit, D_fake_prob = D(x_fake, dropout_rate, is_training,
                                                   reuse = True, print_summary = print_summary)

    return D_real_features, D_real_logit, D_real_prob, D_fake_features, D_fake_logit, D_fake_prob, x_fake

In [0]:
def clip_value(value):
  value = tf.clip_by_value(value, eps_min, eps_max)
  return value

In [0]:
def prepare_labels(label):
    # add extra label for fake data
    extended_label = tf.concat([label, tf.zeros([tf.shape(label)[0], 1])], axis = 1)

    return extended_label

In [0]:
def loss_accuracy(D_real_features, D_real_logit, D_real_prob, D_fake_features,
                  D_fake_logit, D_fake_prob, extended_label, labeled_mask):
    epsilon = 1e-8 # used to avoid NAN loss
    # *** Discriminator loss ***
    # supervised loss
    # which class the real data belongs to
    tmp = tf.nn.softmax_cross_entropy_with_logits(logits = D_real_logit,
                                                  labels = extended_label)
    D_L_supervised = tf.reduce_sum(labeled_mask * tmp) / tf.reduce_sum(labeled_mask) # to ignore
                                                                                     # unlabeled
                                                                                     # data

    # unsupervised loss
    # data is real
    prob_real_be_real = 1 - D_real_prob[:, -1] + epsilon
    tmp_log = tf.log(prob_real_be_real)
    D_L_unsupervised1 = -1 * tf.reduce_mean(tmp_log)

    # data is fake
    prob_fake_be_fake = D_fake_prob[:, -1] + epsilon
    tmp_log = tf.log(prob_fake_be_fake)
    D_L_unsupervised2 = -1 * tf.reduce_mean(tmp_log)

    D_L = D_L_supervised + D_L_unsupervised1 + D_L_unsupervised2

    # *** Generator loss ***
    # fake data is mistaken to be real
    prob_fake_be_real = 1 - D_fake_prob[:, -1] + epsilon
    tmp_log =  tf.log(prob_fake_be_real)
    G_L1 = -1 * tf.reduce_mean(tmp_log)

    # Feature Maching
    tmp1 = tf.reduce_mean(D_real_features, axis = 0)
    tmp2 = tf.reduce_mean(D_fake_features, axis = 0)
    G_L2 = tf.reduce_mean(tf.square(tmp1 - tmp2))

    G_L = G_L1 + G_L2

    # accuracy
    correct_prediction = tf.equal(tf.argmax(D_real_prob[:, :-1], 1),
                                  tf.argmax(extended_label[:, :-1], 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return D_L_supervised, D_L_unsupervised1, D_L_unsupervised2, D_L, G_L, accuracy

In [0]:
def optimizer(D_Loss, G_Loss, D_learning_rate, G_learning_rate):
    # D and G optimizer
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.control_dependencies(extra_update_ops):
        all_vars = tf.trainable_variables()
        D_vars = [var for var in all_vars if var.name.startswith('Discriminator')]
        G_vars = [var for var in all_vars if var.name.startswith('Generator')]

        D_optimizer = tf.train.AdamOptimizer(D_learning_rate).minimize(D_Loss, var_list = D_vars)
        G_optimizer = tf.train.AdamOptimizer(G_learning_rate).minimize(G_Loss, var_list = G_vars)

        return D_optimizer, G_optimizer

Visualize

In [0]:
def plot_fake_data(data, grid_size = [5, 5]):
    # visualize some data generated by G
    _, axes = plt.subplots(figsize = grid_size, nrows = grid_size[0], ncols = grid_size[1],
                           sharey = True, sharex = True)

    size = grid_size[0] * grid_size[1]
    index = np.int_(np.random.uniform(0, data.shape[0], size = (size)))
    figs = data[index].reshape(-1, x_height, x_width, num_channels)

    for idx, ax in enumerate(axes.flatten()):
        ax.axis('off')
        ax.imshow(figs[idx])
    plt.tight_layout()
    plt.show()

In [0]:
import matplotlib.gridspec as gridspec
import cv2

def plot(samples):
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(8, 8)
    gs.update(wspace=0.05, hspace=0.05)
    
    #samples = samples*127.5 + 127.5
    
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample)

    return fig

def im_resize(img, des_side):
    # get height and width of image
    height, width = img.shape[:2]
    
    #For rgb to grayscale
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #sqaure image with side equal to height of rectangle
    img_sq = cv2.resize(img,(des_side,des_side),interpolation = cv2.INTER_CUBIC)
    return img_sq

Train Model

In [0]:
def save_model_on_imporvemnet(file_path, sess, cv_acc, cv_accs):
  #  # save model when there is improvemnet in cv_acc value
    if cv_accs == [] or cv_acc > np.max(cv_accs):
        saver = tf.train.Saver(max_to_keep = 1)
        saver.save(sess, file_path)
        print('Model saved')
    print('')

In [0]:
def get_labled_mask(labeled_rate, batch_size):
    # get labeled mask to mask some data unlabeled
    labeled_mask = np.zeros([batch_size], dtype = np.float32)
    labeled_count = np.int(batch_size * labeled_rate)
    labeled_mask[range(labeled_count)] = 1.0
    #np.random.shuffle(labeled_mask)

    return labeled_mask

In [0]:
def log_loss_acc(file_path, epoch, train_loss_D, train_loss_G, train_Acc,
                 cv_loss_D, cv_loss_G, cv_Acc, log_mode = 'a'):
    # log train and cv losses as well as accuracy
    mode = log_mode if epoch == 0 else 'a'

    with open(file_path, mode) as f:
        if mode == 'w':
            header = 'epoch, train_loss_D, train_loss_G, train_Acc,' \
                     'cv_loss_D, cv_loss_G, cv_Acc\n'
            f.write(header)

        line = '%d, %f, %f, %f, %f, %f, %f\n' %(epoch, train_loss_D, train_loss_G, train_Acc,
                                                cv_loss_D, cv_loss_G, cv_Acc)
        f.write(line)

In [0]:
batch_size = 512
prev_epoch = -1
epochs = 301


#Initialize outputs
train_D_losses, train_G_losses, train_Accs = [], [], []
cv_D_losses, cv_G_losses, cv_Accs = [], [], []

#reset default graph
tf.reset_default_graph()

#Graph inputs
x = tf.placeholder(tf.float32, name = 'x', shape = [None, x_height, x_width, num_channels])
label = tf.placeholder(tf.float32, name = 'label', shape = [None, num_classes])
labeled_mask = tf.placeholder(tf.float32, name = 'labeled_mask', shape = [None])
z = tf.placeholder(tf.float32, name = 'z', shape = [None, latent_size])
dropout_rate = tf.placeholder(tf.float32, name = 'dropout_rate')
is_training = tf.placeholder(tf.bool, name = 'is_training')
G_learning_rate = tf.placeholder(tf.float32, name = 'G_learning_rate')
D_learning_rate = tf.placeholder(tf.float32, name = 'D_learning_rate')


##Define the model
model = build_model(x, z, label, dropout_rate, is_training, print_summary = True)
    
    
D_real_features, D_real_logit, D_real_prob, \
D_fake_features, D_fake_logit, D_fake_prob, \
fake_data = model #

#Results
extended_label = prepare_labels(label)

loss_acc  = loss_accuracy(D_real_features, D_real_logit, D_real_prob,
                          D_fake_features, D_fake_logit, D_fake_prob,
                              extended_label, labeled_mask)

_, _, _, D_L, G_L, accuracy = loss_acc # D_loss, G_loss, accuracy 
D_optimizer, G_optimizer = optimizer(D_L, G_L, G_learning_rate, D_learning_rate) 


# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()


train_images = dataset['train_images']
train_labels = dataset['train_labels']
test_images = dataset['test_images']
test_labels = dataset['test_labels']


# Running first session
print("Training...")
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    if prev_epoch > 0:
      # Restore model weights from previously saved model if any
      saver.restore(sess, model_path)
      print("Model restored at epoch %d from file: %s" %(prev_epoch, model_path))

    for epoch in range(prev_epoch+1, epochs):
      t_total = 0
      train_size = train_labels.shape[0]
      n_iter = int(train_size / batch_size) +  1 
      st_ind = 0
      train_acc = []
      
      for iter in range(int(train_images.shape[0] / batch_size) + 1):
        t_start = time.time()
                
        if iter == n_iter - 1:
            st_ind = train_size - batch_size - 1
            idx = np.random.randint(st_ind, train_size, batch_size)
        else:
            last_ind = st_ind + batch_size
            idx = np.random.randint(st_ind, last_ind, batch_size)
            st_ind += batch_size
        #print(iter, idx)
  
        #idx = np.random.randint(0, train_images.shape[0], batch_size)
        batch_images = train_images[idx]
        batch_labels = train_labels[idx]
        batch = (batch_images, batch_labels)
                
        batch_z = np.random.uniform(-1.0, 1.0, size = (batch_size, latent_size))
        mask = get_labled_mask(labeled_rate, batch_size)
                
        train_feed_dictionary = {x:  normalize(batch[0]),
                                         z: batch_z,
                                         label: batch[1],
                                         labeled_mask: mask,
                                         dropout_rate: 0.2,
                                         G_learning_rate: 0.0002,
                                         D_learning_rate: 0.0002,
                                         is_training: True}

        D_optimizer.run(feed_dict = train_feed_dictionary)
        G_optimizer.run(feed_dict = train_feed_dictionary)

        train_D_loss = D_L.eval(feed_dict = train_feed_dictionary)
        train_G_loss = G_L.eval(feed_dict = train_feed_dictionary)
        train_accuracy = accuracy.eval(feed_dict = train_feed_dictionary)
        t_total += (time.time() - t_start)
        
        train_acc.append(train_accuracy)
        
      train_accuracy = np.mean(train_acc)          
      #pdb.set_trace()
      
      log_loss_acc(log_path, epoch, train_D_loss, train_G_loss, train_accuracy,
                         train_D_loss, train_G_loss, train_accuracy, log_mode = 'w')
      print('epoch: ', epoch, 'time: ',t_total, 'train_G_Loss: ',train_G_loss, 'train_D_Loss: ', 
                  train_D_loss, 'train_acc: ', train_accuracy)
      
      #print("Cross validation!")
      if epoch % 5 == 0:
        # Cross-Validation
        cv_size = test_labels.shape[0]
        n_it = int(cv_size / batch_size) +  1 
        st_ind = 0
        cv_acc = []
      
        for test_it in range(n_it):
          if test_it == n_it - 1:
            st_ind = cv_size - batch_size - 1
            idx = np.random.randint(st_ind, cv_size, batch_size)
          else:
            last_ind = st_ind + batch_size
            idx = np.random.randint(st_ind, last_ind, batch_size)
            st_ind += batch_size
          #print(test_it, idx)
  
          batch_images = test_images[idx]
          batch_labels = test_labels[idx]
          test_batch = (batch_images, batch_labels)
        
          cv_batch_z = np.random.uniform(-1.0, 1.0, size = (batch_size, latent_size))
          mask = get_labled_mask(1, batch_size)
          cv_feed_dictionary = {x: normalize(test_batch[0]),
                                  z: cv_batch_z,
                                  label: test_batch[1],
                                  labeled_mask: mask,
                                  dropout_rate: 0.0,
                                  is_training: False}

          cv_D_loss = D_L.eval(feed_dict = cv_feed_dictionary)
          cv_G_loss = G_L.eval(feed_dict = cv_feed_dictionary)
          cv_accuracy = accuracy.eval(feed_dict = cv_feed_dictionary)
          cv_acc.append(cv_accuracy)
      
        cv_accuracy = np.mean(cv_acc)
      
        
        print('\ncv_G_Loss: %f, cv_D_loss: %f, cv_acc: %f\n' %(cv_G_loss,
                                                                 cv_D_loss,
                                                                 cv_accuracy))
            
        #save_model_on_imporvemnet(model_path, sess, cv_accuracy, cv_Accs)
            
      
        #if cv_Accs == [] or cv_accuracy > np.max(cv_Accs):
        save_path = saver.save(sess, model_path)
        print("Model saved in file: %s" % save_path)
        
        fakes = fake_data.eval(feed_dict = train_feed_dictionary)
        rescaled_fakes = (fakes + 1.0)/2.0
        #plot_fake_data(rescaled_fakes, [8, 8])
        fig = plot(rescaled_fakes[:64])
        plt.savefig((save_to + 'G@epoch{}.png').format(str(epoch).zfill(3)), bbox_inches = "tight")
        plt.close()
      
        cv_D_losses.append(cv_D_loss)
        cv_G_losses.append(cv_G_loss)
        cv_Accs.append(cv_accuracy)

Discriminator summary:
 x: (?, 32, 32, 3)
 D1: (?, 16, 16, 64)
 D2: (?, 8, 8, 128)
 D3: (?, 4, 4, 256)
 D4: (?, 2, 2, 512)

(?, 32, 32, 3)
Generator summary:
 z: (?, 2048)
 G0: (?, 2, 2, 512)
 G1: (?, 4, 4, 256)
 G2: (?, 8, 8, 128)
 G3: (?, 16, 16, 64)
 G4: (?, 32, 32, 3)

Discriminator summary:
 x: (?, 32, 32, 3)
 D1: (?, 16, 16, 64)
 D2: (?, 8, 8, 128)
 D3: (?, 4, 4, 256)
 D4: (?, 2, 2, 512)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Training...
epoch:  0 time:  186.51557111740112 train_G_Loss:  3.7119656 train_D_Loss:  2.0855513 train_acc:  0.20540364

cv_G_Loss: 0.249586, cv_D_loss: 7.821461, cv_acc: 0.191598

Model saved in file: gdrive/My Drive/gan/svhn/model_Jan16/gan_svhn_model.ckpt
epoch:  1 time:  181.76082468032837 train_G_Loss:  2.343976 train_D_Loss:  2.1037934 train_acc:  0.32881674
epoch:  2 time:  181.5683605670929 train_G_